# GWAS
We're going to run GWAS using the Limix library for python.
Limix is freely available [here](https://github.com/limix/limix) and has an extensive [documentation](https://limix.readthedocs.io/).

## Set up the environment


In [1]:
import pandas as pd
import numpy as np
import os
import h5py
from limix.qtl import scan
from bisect import bisect

## settings

In [2]:
# phenotype
pheno_file = './data/flowering_time_16.csv'
# genotype
geno_file = './data/1001genomes/all_chromosomes_binary.hdf5'
# kinship matrix
kin_file = './data/1001genomes/kinship_ibs_binary_mac5.h5py'
# minor allele frequency threshold
MAF_thrs = 0.1
# output results
output_file = './data/flowering_time_16_gwas.csv'

## Load phenotype
The phenotype data is stored in a 2-columns .csv file.
The first column specifies the genotype ID, the seconc columns contains the phenotype.

In [3]:
pheno = pd.read_csv(pheno_file, index_col = 0)
# remove NA values
pheno = pheno[np.isfinite(pheno)]
# encode the index to UTF8 for compatability with the genotype data
pheno.index = pheno.index.map(lambda x: str(int(x)).encode('UTF8'))

## Load genotype
The genotype file we're going to use is the SNP-matrix obtained from whole genome sequencing 1,135 Arabidopsis thaliana accessions ([1001 genomes](http://1001genomes.org/)).
The SNP-matrix is stored as an [hdf5](https://www.h5py.org/) file.


In [4]:
geno_hdf = h5py.File(geno_file, 'r')

In [7]:
# remove non-genotyped accessions from phenotype
acn_genotyped = [acn for acn in pheno.index if acn in geno_hdf['accessions'][:]]
# subset phenotype data
pheno = pheno.loc[acn_genotyped]
# order genotypes in phenotype according SNP-matrix
acn_indices = [np.where(geno_hdf['accessions'][:] == acn)[0][0] for acn in pheno.index]
acn_indices.sort()
acn_order = geno_hdf['accessions'][acn_indices]
pheno = pheno.loc[acn_order]
# transform phenotype matrix (Y)
Y = pheno.to_numpy()

In [8]:
# subset SNP-matrix for phenotyped genotypes
G = geno_hdf['snps'][:, acn_indices]

In [11]:
# remove SNPs with minor allele frequency below set threshold
# count allele 1 and 0 for each SNP
AC1 = G.sum(axis = 1)
AC0 = G.shape[1] - AC1
AC = np.vstack((AC0,AC1))
# define the minor allele for each position
MAC = np.min(AC, axis = 0)
# calculate minor allele frequency
MAF = MAC/G.shape[1]
# select SNPs with MAF above threshold 
SNP_indices = np.where(MAF >= MAF_thrs)[0]
SNPs_MAF = MAF[SNP_indices]
G = G[SNP_indices, :]

# transpose SNP-matrix into accessions x SNPs matrix
G = G.transpose()
geno_hdf.close()

## Load kinship matrix

In [12]:
kin_hdf = h5py.File(kin_file, 'r')

In [13]:
# subset kinship matrix for phenotyped and genotyped accessions
acn_indices = [np.where(kin_hdf['accessions'][:] == acn)[0][0] for acn in pheno.index]
acn_indices.sort()
K = kin_hdf['kinship'][acn_indices, :][:, acn_indices]
kin_hdf.close()

## GWAS

In [14]:
r = scan(G, Y, K = K, lik = 'normal', M = None, verbose = True)

============================= QTL analysis session starts ==============================

Normalising input... 
done (6.2 seconds).


LMM: 30it [00:00, 458.41it/s]
Results: 100%|██████████| 953/953 [00:00<00:00, 40453.11it/s]


Hypothesis 0

𝐲 ~ 𝓝(𝙼𝜶, 5636.410⋅𝙺 + 35.535⋅𝙸)

M     = ['offset']
𝜶     = [82.17817371]
se(𝜶) = [72.43939806]
lml   = -4035.8809844110856

Hypothesis 2

𝐲 ~ 𝓝(𝙼𝜶 + G𝛃, s(5636.410⋅𝙺 + 35.535⋅𝙸))

          lml       cov. effsizes   cand. effsizes
--------------------------------------------------
mean   -4.036e+03       8.229e+01       -2.302e-01
std     3.273e-01       2.464e-01        1.321e+00
min    -4.036e+03       8.156e+01       -1.377e+01
25%    -4.036e+03       8.216e+01       -7.399e-01
50%    -4.036e+03       8.226e+01       -2.522e-01
75%    -4.036e+03       8.243e+01        3.965e-01
max    -4.032e+03       8.330e+01        6.103e+00

Likelihood-ratio test p-values

       𝓗₀ vs 𝓗₂ 
----------------
mean   5.948e-01
std    2.403e-01
min    3.263e-03
25%    3.936e-01
50%    6.178e-01
75%    8.053e-01
max    9.983e-01


/Users/pieter.clauw/anaconda3/envs/limix/lib/python3.7/site-packages/limix/qtl/_result/_st_result.py:115: FutureWarning: `item` has been deprecated and will be removed in a future version
  v0 = self.h0.variances["fore_covariance"].item()
/Users/pieter.clauw/anaconda3/envs/limix/lib/python3.7/site-packages/limix/qtl/_result/_st_result.py:116: FutureWarning: `item` has been deprecated and will be removed in a future version
  v1 = self.h0.variances["back_covariance"].item()


====================== QTL analysis session ends in 8.28 seconds =======================

In [15]:
# save results
# link chromosome and position to p-values and effect sizes
geno_hdf = h5py.File(geno_file, 'r')
chr_index = geno_hdf['positions'].attrs['chr_regions']
chromosomes = [bisect(chr_index[:, 1], snp_index) + 1 for snp_index in SNP_indices]
positions_all = geno_hdf['positions'][:]
positions = [positions_all[snp] for snp in SNP_indices]
pvalues = r.stats.pv20.tolist()
effsizes = r.effsizes['h2']['effsize'][r.effsizes['h2']['effect_type'] == 'candidate'].tolist()

gwas_results = pd.DataFrame(list(zip(chromosomes, positions, pvalues, SNPs_MAF, MAC[SNP_indices], effsizes)), columns = ['chr', 'pos', 'pvalue', 'maf', 'mac', 'GVE'])
gwas_results.to_csv(output_file, index = False)
geno_hdf.close()
